Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 4 products, then either $y=3$ or $y=(0,0,1,0)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, size, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 4 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta} + e^{x_4'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=\delta_{i4}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 \times \mathbb{P}_i(4)^0 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$


We will use the `yogurt_data` dataset, which provides anonymized consumer identifiers (`id`), a vector indicating the chosen product (`y1`:`y4`), a vector indicating if any products were "featured" in the store as a form of advertising (`f1`:`f4`), and the products' prices (`p1`:`p4`). For example, consumer 1 purchased yogurt 4 at a price of 0.079/oz and none of the yogurts were featured/advertised at the time of consumer 1's purchase.  Consumers 2 through 7 each bought yogurt 2, etc.

_todo: import the data, maybe show the first few rows, and describe the data a bit._

In [9]:
import pandas as pd

yogurt = pd.read_csv('yogurt_data.csv')

yogurt.head()

,id,y1,y2,y3,y4,f1,f2,f3,f4,p1,p2,p3,p4
0,1,0,0,0,1,0,0,0,0,0.108,0.081,0.061,0.079
1,2,0,1,0,0,0,0,0,0,0.108,0.098,0.064,0.075
2,3,0,1,0,0,0,0,0,0,0.108,0.098,0.061,0.086
3,4,0,1,0,0,0,0,0,0,0.108,0.098,0.061,0.086
4,5,0,1,0,0,0,0,0,0,0.125,0.098,0.049,0.079


In [10]:
yogurt.shape

(2430, 13)

In [11]:
yogurt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2430 entries, 0 to 2429
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      2430 non-null   int64  
 1   y1      2430 non-null   int64  
 2   y2      2430 non-null   int64  
 3   y3      2430 non-null   int64  
 4   y4      2430 non-null   int64  
 5   f1      2430 non-null   int64  
 6   f2      2430 non-null   int64  
 7   f3      2430 non-null   int64  
 8   f4      2430 non-null   int64  
 9   p1      2430 non-null   float64
 10  p2      2430 non-null   float64
 11  p3      2430 non-null   float64
 12  p4      2430 non-null   float64
dtypes: float64(4), int64(9)
memory usage: 246.9 KB


In [5]:
yogurt.describe()

,id,y1,y2,y3,y4,f1,f2,f3,f4,p1,p2,p3,p4
count,2430.0000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000
mean,1215.5000,0.341975,0.401235,0.029218,0.227572,0.055556,0.039506,0.037449,0.037449,0.106248,0.081532,0.053622,0.079507
std,701.6249,0.474469,0.490249,0.168452,0.419351,0.229109,0.194836,0.189897,0.189897,0.020587,0.011047,0.008054,0.007714
min,1.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.012000,0.000000,0.025000,0.004000
25%,608.2500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103000,0.081000,0.050000,0.079000
50%,1215.5000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.108000,0.086000,0.054000,0.079000
75%,1822.7500,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.115000,0.086000,0.061000,0.086000
max,2430.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.193000,0.111000,0.086000,0.104000


After conducting some research on the dataset, it is shown that the rate at which products y1 through y4 are selected are between 2% and 40%, with 2% being extremely low compared to the other products (23%, 34%). Each product also has a large standard deviation of being selected, likely because the product is either selected (1) or not selected (0). Also, the averages of each product being selected sum to 1, meaning there is no missing data for the y columns in the dataset.

The products were featured approximately same proportion of time (between 3.7%-5.6%), with product 1 being featured more than others at 5.6%, but it was not the most selected product. That product is y2. The summed average of featured products does not sum to 1, meaning a product wasn't always featured. The standard deviation are also very large, likely for the same reason as mentioned above (binary). They are slightly higher SD's proportionally to the averages, which is likely because there are much more 0's than 1's in columns due to the fact that there doesn't have to be a product featured. 

Average prices per ounce range from approximately $0.11 to $0.05. These standard deviations are not nearly as proportionally high as the other columns, likely because the prices are continuous and not binary. Price seems to potentially follow a normal distribution.

Let the vector of product features include brand dummy variables for yogurts 1-3 (we'll omit a dummy for product 4 to avoid multi-collinearity), a dummy variable to indicate if a yogurt was featured, and a continuous variable for the yogurts' prices:  

$$ x_j' = [\mathbf{1}_{\text{Yogurt 1}}, \mathbf{1}_{\text{Yogurt 2}}, \mathbf{1}_{\text{Yogurt 3}}, X_f, X_p] $$


The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). 

What we would like to do is reorganize the data from a "wide" shape with $n$ rows and multiple columns for each covariate, to a "long" shape with $n \times J$ rows and a single column for each covariate.  As part of this re-organization, we'll add binary variables to indicate the first 3 products; the variables for featured and price are included in the dataset and simply need to be "pivoted" or "melted" from wide to long.  

_todo: reshape and prep the data_

In [26]:
# Melting and restructuring the data in one step
long_data = yogurt.melt(id_vars='id', 
                             value_vars=['y1', 'y2', 'y3', 'y4', 'f1', 'f2', 'f3', 'f4', 'p1', 'p2', 'p3', 'p4'],
                             var_name='variable', 
                             value_name='value')

# Adding separate columns for type and product number directly
long_data['type'] = long_data['variable'].str[0]
long_data['product_num'] = long_data['variable'].str[1].astype(int)

# Pivoting to have separate columns for each data type (purchase, featured, price)
long_data = long_data.pivot_table(index=['id', 'product_num'], columns='type', values='value', aggfunc='first').reset_index()

# Renaming the columns for clarity
long_data.columns = ['id', 'product_num', 'featured', 'price', 'purchase']

# Creating binary indicators for the first three products
long_data['yogurt_1'] = (long_data['product_num'] == 1).astype(int)
long_data['yogurt_2'] = (long_data['product_num'] == 2).astype(int)
long_data['yogurt_3'] = (long_data['product_num'] == 3).astype(int)


In [27]:
long_data

,id,product_num,featured,price,purchase,yogurt_1,yogurt_2,yogurt_3
0,1,1,0.0,0.108,0.0,1,0,0
1,1,2,0.0,0.081,0.0,0,1,0
2,1,3,0.0,0.061,0.0,0,0,1
3,1,4,0.0,0.079,1.0,0,0,0
4,2,1,0.0,0.108,0.0,1,0,0
...,...,...,...,...,...,...,...,...
9715,2429,4,0.0,0.086,1.0,0,0,0
9716,2430,1,0.0,0.108,0.0,1,0,0
9717,2430,2,0.0,0.086,0.0,0,1,0
9718,2430,3,0.0,0.043,0.0,0,0,1


_todo: Code up the log-likelihood function._

In [28]:
import numpy as np

def multinomial_log_likelihood(X, y, beta):
    """
    Calculate the log-likelihood for multinomial logistic regression.

    Parameters:
    - X (list of numpy arrays): List where each element is a feature matrix for one category
    - y (numpy array): Vector of observed category indices (0-based)
    - beta (numpy array): Matrix of coefficients for all categories (except the base category)

    Returns:
    - log_likelihood (float): The computed log-likelihood value
    """
    # Calculate linear combinations for each category
    linear_combinations = [np.dot(X_j, beta_j) for X_j, beta_j in zip(X, beta)]

    # Exponentiate the linear combinations
    exp_terms = [np.exp(lin_comb) for lin_comb in linear_combinations]

    # Calculate the denominator (sum of exponentiated terms for all categories)
    denom = sum(exp_terms)

    # Log of probabilities for the chosen category for each observation
    log_probs = [np.log(exp_term[i] / denom[i]) for i, exp_term in enumerate(exp_terms)]

    # Calculate the log-likelihood by summing log probabilities for the observed categories
    log_likelihood = sum(log_probs[i] for i in y)

    return log_likelihood


_todo: Use `optim()` in R or `optimize()` in Python to find the MLEs for the 5 parameters ($\beta_1, \beta_2, \beta_3, \beta_f, \beta_p$).  (Hint: you should find 2 positive and 1 negative product intercepts, a small positive coefficient estimate for featured, and a large negative coefficient estimate for price.)_